In [1]:
# Ensure tf-keras is installed
%pip install tf-keras
%pip install setuptools
%pip install opencv-python 
%pip install deepface

  Using cached tf_keras-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp312-cp312-win_amd64.whl.metadata (4.9 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached protobuf-5.29.3-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached setuptools-75.8.0-py3-none-any.whl.metadata (6.7 kB)
  Using ca

In [3]:
import cv2 as cv
import setuptools.dist
from deepface import DeepFace
import numpy as np
from PIL import Image
import threading
import os

os.environ['DEEPFACE_HOME'] = 'C:/Users/YASHASWAT/Desktop/FaceDetect/' # env variable to download model weights in current folder

In [5]:
DeepFace.build_model("Facenet")
DeepFace.build_model("VGG-Face")

25-03-03 20:42:59 - facenet_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facenet_weights.h5
To: C:\Users\YASHASWAT\Desktop\FaceDetect\.deepface\weights\facenet_weights.h5
100%|██████████| 92.2M/92.2M [00:05<00:00, 18.3MB/s]


25-03-03 20:43:07 - vgg_face_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5
To: C:\Users\YASHASWAT\Desktop\FaceDetect\.deepface\weights\vgg_face_weights.h5
100%|██████████| 580M/580M [00:34<00:00, 16.7MB/s] 


### Webcam Implementation

In [11]:
# webcam live feed and capturing images from that
# WORKING

# starting webcam feed
camera = cv.VideoCapture(0)

# loading reference image for verification
reference_img = cv.imread('Verified Faces/yash_face2.jpg')

frame_no = 0 # counter
face_match = False # face match flag
model = 'VGG-Face' # model to use for face recognition
threshold = 0.4 # threshold for embedding distance

# function checks if webcam frame matches reference image
# updates face_match flag
def check_face(frame, model):
    global face_match
    global reference_img
    
    try:
        if DeepFace.verify(frame, reference_img.copy(), model_name='model', 
                           detector_backend='opencv', align=True, threshold=threshold)['verified']:
            face_match = True
        else:
            face_match = False
    except ValueError:
        face_match = False

# main loop
if not camera.isOpened():
    print('couldnt open cam')
    exit()
else:
    
    while True:
        
        ret, frame = camera.read() # read frame from webcam
        
        if ret:
            
            # run check_face function every 30 frames
            if frame_no % 30 == 0:
                try:
                    threading.Thread(target=check_face, args=(frame.copy(), model)).start()
                except ValueError:
                    pass
            
            frame_no += 1
            
            # display match status on webcam feed
            if face_match:
                cv.putText(frame, 'MATCH!', (50, 50), cv.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2, cv.LINE_AA)
            else:
                cv.putText(frame, 'NO MATCH!', (50, 50), cv.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2, cv.LINE_AA)

            cv.imshow('face video', frame)
        
        # key press handling
        key = cv.waitKey(1) & 0xFF        
        
        if key == ord('c'): # capture and save frame
                        
            cv.imwrite(f'Snapshots/frame{frame_no}.jpg', frame)
            frame_no += 1
            continue
        
        if key == ord('q'): # quit
            break
    
camera.release()
cv.destroyAllWindows()

### Testing Code

In [ ]:
img1_path = 'Verified Faces/yash_nospecs.jpg'
img2_path = 'Verified Faces/yash_pport.jpg'

img1 = cv.imread(img1_path)
img2 = cv.imread(img2_path)

# using verify function to fetch face dimensions for both images
result = DeepFace.verify(img1_path=img1_path, img2_path=img2_path, model_name='VGG-Face')

img1_dims = result['facial_areas']['img1']
img2_dims = result['facial_areas']['img2']

# cropping images to only the face dimensions
img1 = img1[img1_dims['y']:img1_dims['y']+img1_dims['h'], img1_dims['x']:img1_dims['x']+img1_dims['w']]
img2 = img2[img2_dims['y']:img2_dims['y']+img2_dims['h'], img2_dims['x']:img2_dims['x']+img2_dims['w']]

# displaying cropped face images
cv.imshow('yash', img1)
cv.waitKey(0)
cv.destroyAllWindows()

cv.imshow('yogs', img2)
cv.waitKey(0)
cv.destroyAllWindows()

# verifying the cropped images with specified threshold
new_result = DeepFace.verify(img1, img2, model_name='VGG-Face', threshold=0.4)

print(new_result)

{'verified': False, 'distance': 0.9660290783855201, 'threshold': 0.4, 'model': 'VGG-Face', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 0, 'y': 0, 'w': 330, 'h': 330, 'left_eye': None, 'right_eye': None}, 'img2': {'x': 22, 'y': 13, 'w': 389, 'h': 389, 'left_eye': (278, 164), 'right_eye': (137, 166)}}, 'time': 0.54}
